#### 문제 설명  
처리해야 할 동일한 작업이 n 개가 있고, 이를 처리하기 위한 CPU가 있습니다.  
  
이 CPU는 다음과 같은 특징이 있습니다.  
  
- CPU에는 여러 개의 코어가 있고, 코어별로 한 작업을 처리하는 시간이 다릅니다.
- 한 코어에서 작업이 끝나면 작업이 없는 코어가 바로 다음 작업을 수행합니다.
- 2개 이상의 코어가 남을 경우 앞의 코어부터 작업을 처리 합니다.  
  
처리해야 될 작업의 개수 n과, 각 코어의 처리시간이 담긴 배열 cores 가 매개변수로 주어질 때, 마지막 작업을 처리하는 코어의 번호를 return 하는 solution 함수를 완성해주세요.  
   
#### 제한 조건  
- 코어의 수는 10,000 이하 2이상 입니다.
- 코어당 작업을 처리하는 시간은 10,000이하 입니다.
- 처리해야 하는 일의 개수는 50,000개를 넘기지 않습니다.

#### 입출력 예  
|n|	cores|	result|
|:---:|:---:|:---:|
|6|	[1,2,3]	|2  |

---

### 문제 풀이

In [1]:
# 우선순위 큐를 이용한 풀이 -> 시간초과
from queue import PriorityQueue

def solution(n, cores):
    q = PriorityQueue()
    for i in range(len(cores)): # 모든 코어에 대해서
        if i == n:
            return i+1 # 해야할 작업의 개수가 코어 총 개수보다 작을 때는 차례에 따른 인덱스 반환
        q.put((cores[i], i)) # (코어의 총 작업시간, 코어의 인덱스) 정보를 큐에 push
    job_done = len(cores) # 이미 수행한 작업의 개수
    while not q.empty():
        time, idx = q.get() # (코어의 총 작업시간, 코어의 인덱스) 정보를 큐에서 pop
        job_done += 1 # 수행한 작업개수 1 증가
        if n == job_done: # 만약 수행한 작업개수가 n개라면
            return idx+1 # 해당 코어 인덱스 반환
        q.put((time+cores[idx], idx)) # (코어의 총 작업시간 + 현재 코어의 작업시간, 코어의 인덱스) 정보를 다시 큐에 push
    return -1

In [2]:
# paramatic search를 이용한 풀이 -> 성공

def solution(n, cores):
    answer= 0 # 답
    core_num = len(cores) # 코어 개수
    min_core = min(cores) # 코어 소모시간 중에 최소시간
    max_core = max(cores) # 코어 소모시간 중에 최대시간
    min_time = (min_core * n) // core_num - min_core # min_time을 소모하는 코어로만 이루어진 cpu에서 n개의 작업을 해내는데 걸리는 시간
    max_time = (max_core * n) // core_num - max_core # max_time을 소모하는 코어로만 이루어진 cpu에서 n개의 작업을 해내는데 걸리는 시간

    while min_time <= max_time:
        mid_time = (min_time + max_time) // 2 # min_time과 max_time의 중간값
        
        done = core_num # mid_time동안 수행한 작업의 개수
        current_assigned = 0 # mid_time 시각에 바로 할당되는 작업의 개수
        
        for i in range(core_num): # 모든 코어에 대해서
            val = cores[i]
            done += mid_time // val # 수행한 작업들을 더한다.
            if mid_time % val == 0: # 해당 코어에서 마침 작업을 마쳤을 때
                current_assigned += 1 # 바로 할당될 작업의 개수를 1 증가시킨다.
        
        
        if n > done: # 만약 이미 수행한 작업들이 n개보다 작을 때 
            min_time = mid_time+1 # min_time을 중간시간+1로 갱신
        elif n <= done-current_assigned: # 지금 바로 할당된 작업을 제외한 수행 작업들이 n개 이상일 때
            max_time = mid_time-1 # max_time을 중간시간-1로 갱신
        else: # done-assigned_now < n <= done
            # 이미 끝마친 작업들을 제외하고, 새로 할당된 작업들 중에서
            count = 0
            for i in range(core_num):
                if mid_time % cores[i] == 0:
                    count += 1
                if count == n - (done - current_assigned): # 마지막에 할당될 작업일 때
                    return i+1 # 해당 작업의 인덱스를 반환
    return answer

In [3]:
n = 6
cores = [1,2,3]
solution(n, cores)

2